In [1]:
import json
import pandas as pd
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import GridspecLayout
from markdown_it import MarkdownIt

In [2]:
# Custom CSS to ensure text wraps properly
css = """
<style>
.custom-button {
    white-space: normal;
    font-size: 16px !important;
    font-weight: bold !important;
}
</style>
"""

In [3]:
# Custom CSS to ensure text wraps properly
header_css = """
<style>
.custom-header {
    white-space: normal;
    font-size: 18px !important;
    font-weight: bold !important;
    padding: 50px;
    color: #000000
}
</style>
"""

In [4]:
# Custom CSS to ensure text wraps properly
footer_css = """
<style>
.custom-footer {
    width: auto;
    align-item: flex-start !important; 
    padding-top: 70px;
    height: auto
}
</style>
"""

In [5]:
# Header widget
html_header = widgets.HTML(value="<h1>Remote Camera Decision Support Tool</h1>")

In [6]:
html_header.add_class("custom-header")
# Create a HTML widget to include the custom CSS
header_css_widget = widgets.HTML(value=header_css)

In [8]:
html_footer = widgets.HTML(value="""
<footer>
    <img src='footer.png' alt='Footer Image'>
</footer>
""")


In [9]:
html_footer.add_class("custom-footer")
# Create a HTML widget to include the custom CSS
footer_css_widget = widgets.HTML(value=footer_css)

In [10]:
# Image Widget

file = open("../logo.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='200'
                )

label_headline = widgets.Label(
                    value='',
                    style={'description_width': 'initial'}
                )

# vbox_headline = widgets.HBox([image_headline, label_headline])

In [11]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto')) #width='max-content'

In [12]:
# Concept Library

concept_header = create_expanded_button('Concept Library', 'success')
concept_header.style.button_color = '#2D6EB4'
concept_header.style.text_color = 'black'

concept_option1 = create_expanded_button('option-1', 'success')
concept_option1 .style.button_color = '#97C3F2'
concept_option1 .style.text_color = 'black'

concept_option2 = create_expanded_button('option-2', 'success')
concept_option2.style.button_color = '#B4D5F8'
concept_option2.style.text_color = 'black'

concept_option3 = create_expanded_button('option-3', 'success')
concept_option3.style.button_color = '#B4D5F9'
concept_option3.style.text_color = 'black'

concept_grid = GridspecLayout(4, 41, grid_gap='5px', height='170px')
concept_grid[0, 1:41] = concept_header
concept_grid[1, 1:41] = concept_option1
concept_grid[2, 1:41] = concept_option2
concept_grid[3, 1:41] = concept_option3
# concept_grid

In [13]:
left_col = widgets.VBox(
    [image_headline, concept_grid],
    layout=widgets.Layout(
        width='auto',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border_right='solid 4px grey',
        padding='10px',
#         margin='auto',  # Center the VBox horizontally
        align_items='flex-start',  # Center the items inside the VBox   "flex-start"
        justify_content='center'  # Center the items vertically inside the VBox
    )
)


# vbox_headline = widgets.VBox([image_headline, concept_grid])
# display(left_col) 

In [14]:
global stored_choices, path

stored_choices = {}
path = "../temp_storage.json"
back_path = "../backtracking.json"

In [15]:
with open(path, "r") as f:
    stored_choices = json.load(f)

In [16]:
with open(back_path, 'r') as f:
    back_list = json.load(f)

In [18]:
# Read the CSV file into a DataFrame

df = pd.read_csv('../new_format.csv', low_memory=False)

In [19]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list

def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [20]:
# gets the question based on value of other column

ques = df.loc[df['question_code'] == 'sp_asymptote', 'question_text'].iloc[0]
opt = df.loc[df['question_code'] == 'sp_asymptote', 'q_option_label'].iloc[0]
opt_codes = df.loc[df['question_code'] == 'sp_asymptote', 'q_option_code'].iloc[0]

opt = split_and_trim(opt)
opt_codes = split_and_trim(opt_codes)

In [62]:
# ques

In [61]:
# Factor widgets

question = create_expanded_button(ques, 'success')
question.style.button_color = 'lightblue'
question.style.text_color = 'black'
# Add custom class to the button
question.add_class('custom-button')

option1 = create_expanded_button(opt[0], 'success')
option1.style.button_color = 'lightgreen'
option1.style.text_color = 'black'

option2 = create_expanded_button(opt[1], 'success')
option2.style.button_color = 'green'
option2.style.text_color = 'black'

option3 = create_expanded_button(opt[2], 'success')
option3.style.button_color = 'green'
option3.style.text_color = 'black'

# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=css)

factor_grid = GridspecLayout(4, 12, grid_gap='3px', height='250px')
factor_grid[1, 1:6] = question
factor_grid[0, 7:9] = option1
factor_grid[1, 7:9] = option2
factor_grid[2, 7:9] = option3


logical_integer_container = widgets.VBox(children=[factor_grid, css_widget], layout=widgets.Layout(
    align_items='center'  # Center align vertically
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))
# logical_integer_container

In [46]:
output = widgets.Output()
def option_1_button_clicked(event):
    with output:
        clear_output()
        global selected_objective
        
        selected_option = factor_grid[0, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["sp_asymptote"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
        selected_objective = stored_choices['objective']
        
factor_grid[0, 7:9].on_click(option_1_button_clicked)

In [47]:
output = widgets.Output()
def option_2_button_clicked(event):
    with output:
        clear_output()
        global selected_objective
        
        selected_option = factor_grid[1, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["sp_asymptote"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        selected_objective = stored_choices['objective']
        
factor_grid[1, 7:9].on_click(option_2_button_clicked)

In [48]:
output = widgets.Output()
def option_3_button_clicked(event):
    with output:
        clear_output()
        global selected_objective
        
        selected_option = factor_grid[2, 7:9].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        stored_choices["sp_asymptote"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        selected_objective = stored_choices['objective']
        
factor_grid[2, 7:9].on_click(option_3_button_clicked)

In [49]:
# Define the iframe HTML for a YouTube video
iframe = """
<iframe width="800" height="500" src="https://ab-rcsc.github.io/RCSC-WildCAM_Remote-Camera-Survey-Guidelines-and-Metadata-Standards/2_metadata-standards/2_8.0_Deployment.html#deployment-start-date-time-dd-mmm-yyyy-hh-mm-ss" 
        frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" 
        allowfullscreen></iframe>
"""

# Display the iframe
# display(HTML(iframe))


In [50]:
# Create an Output widget for the pop-up window content
output_popup = widgets.Output()

# Function to show the pop-up window
def show_popup(b):
    with output_popup:
        output_popup.clear_output()
        display(popup_window)

# Function to close the pop-up window
def close_popup(b):
    output_popup.clear_output()

# Create a close button for the pop-up window
close_button = widgets.Button(
    description="Close",
    layout=widgets.Layout(width='100px')
)
close_button.on_click(close_popup)

In [51]:
# Create the pop-up window
popup_window = widgets.VBox(
    children=[
        widgets.HTML(iframe),
#         widgets.HTML('<h3>Pop-up Window</h3>'),
        close_button
    ],
    layout=widgets.Layout(
        display='flex',
        flex_flow='column',
        align_items='center',
        justify_content='center',
        border='solid 1px grey',
        padding='10px',
        width='auto',
        height='auto',
        background_color='lightyellow'
    )
)

In [52]:
# Create a button that will be positioned at the top right corner
button = widgets.Button(
    description="Show Popup"
#     layout=widgets.Layout(width='100px')
)
button.on_click(show_popup)

In [53]:
# Create a buttons for submitting the question

submit_button = widgets.Button(description="Submit")

In [54]:
# Create a button for going back to previous question

back_button = widgets.Button(description="Back")

In [55]:
# Create a button for skipping the question
skip_button = widgets.Button(description="Skip")
# skip_button.add_class("lbl_bg");

In [56]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(link)

back_button.on_click(on_back_button_clicked)

In [57]:
output = widgets.Output()
def on_skip_button_clicked(event):
    with output:
        clear_output()
        link = Markdown('''<a href="https://voila.readthedocs.io/en/stable/deploy.html" target="_self"> 
                                    Next 
                            </a>''')
        display(link)

skip_button.on_click(on_skip_button_clicked)
# vbox_result = widgets.VBox([question_widget1, options_widget1, submit_button, output, choice])

In [58]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()
        global selected_option1
        next_notebook = ''
        link = "Submit your answer for next question"

        next_notebook = "obj_targ_sp"
            
        notebook_filename = next_notebook + ".ipynb"
        
        link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Next</a>'''.format(notebook_filename))

        back_list.append("sp_asymptote")
        with open (back_path, "w")as f:
            json.dump(back_list, f)

        display(link)

submit_button.on_click(on_button_clicked)

In [59]:

all_button = widgets.HBox(
    [back_button, submit_button, skip_button],
    layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border='solid 2px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    )
)
buttons = widgets.VBox(
    [output, all_button],
    layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border='solid 2px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    )
)



upper = widgets.VBox(
    [logical_integer_container, buttons],
    layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
        border='solid 2px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    )
)


popup = widgets.VBox(children=[button, output_popup], layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
#         border='solid 4px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'  # Center the items vertically inside the VBox
    ))

right_col = widgets.VBox(children=[html_header, header_css_widget, upper, popup, html_footer, footer_css_widget], layout=widgets.Layout(
        width='70%',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
        border_left='solid 5px grey',
        padding='10px',
        margin='auto',  # Center the VBox horizontally
        align_items='center',  # Center the items inside the VBox
        justify_content='center'
))

cols = widgets.HBox(
    [left_col, right_col],
    layout=widgets.Layout(
        width='auto',  # Set the width to 50% of the container or viewport
        height='auto',  # Set a fixed height
        border='solid 4px grey',
#         padding='10px',
#         margin='auto',  # Center the VBox horizontally
        align_items='flex-start',  # Center the items inside the VBox
        justify_content='center',  # Center the items vertically inside the VBox
        background_color='lightyellow'
    )
)

# Display the container

# right_col.add_class("box_style")
# display(left_col)
# display(right_col)
display(cols)
# display(popup)